### Baseline model
hidden_layer_sizes=(256, 128), <br> activation='relu', <br> solver='adam', <br> alpha=1e-4, <br> batch_size=128, <br> learning_rate='adaptive', <br> learning_rate_init=1e-3, <br> max_iter=400, <br> early_stopping=True

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_curve, auc, classification_report

In [2]:
train_df = pd.read_csv('../../../dataset/open_world/openworld_train.csv')
test_df = pd.read_csv('../../../dataset/open_world/openworld_test.csv')

In [3]:
# Feature / Target Split
X = train_df.drop(columns=["label"]).values
y = train_df["label"].values

X_test = test_df.drop(columns=["label"]).values
y_test = test_df["label"].values

# Train / Validation Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [4]:
mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128),
    activation='relu',
    solver='adam',
    alpha=1e-4,
    batch_size=128,
    learning_rate='adaptive',
    learning_rate_init=1e-3,
    max_iter=400,
    early_stopping=True,
    validation_fraction=0.15,
    n_iter_no_change=15,
    random_state=42,
    verbose=True
)

In [5]:
print("\n[INFO] Training MLP (256-128) on Open-world dataset...")
mlp.fit(X_train, y_train)


[INFO] Training MLP (256-128) on Open-world dataset...
Iteration 1, loss = 3.39306217
Validation score: 0.364122
Iteration 2, loss = 2.62049841
Validation score: 0.426108
Iteration 3, loss = 2.28847995
Validation score: 0.460591
Iteration 4, loss = 2.11910486
Validation score: 0.472496
Iteration 5, loss = 1.99083459
Validation score: 0.498358
Iteration 6, loss = 1.88876947
Validation score: 0.504516
Iteration 7, loss = 1.80398505
Validation score: 0.524631
Iteration 8, loss = 1.72404954
Validation score: 0.527504
Iteration 9, loss = 1.66075334
Validation score: 0.543103
Iteration 10, loss = 1.60159090
Validation score: 0.550082
Iteration 11, loss = 1.54422921
Validation score: 0.559524
Iteration 12, loss = 1.49226115
Validation score: 0.575123
Iteration 13, loss = 1.44778216
Validation score: 0.569376
Iteration 14, loss = 1.40193709
Validation score: 0.581691
Iteration 15, loss = 1.36310495
Validation score: 0.591133
Iteration 16, loss = 1.33300444
Validation score: 0.600985
Iteration

MLPClassifier(batch_size=128, early_stopping=True,
              hidden_layer_sizes=(256, 128), learning_rate='adaptive',
              max_iter=400, n_iter_no_change=15, random_state=42,
              validation_fraction=0.15, verbose=True)

In [6]:
# Val
val_pred = mlp.predict(X_val)
val_acc = accuracy_score(y_val, val_pred)
val_f1_macro = f1_score(y_val, val_pred, average='macro')
val_f1_weighted = f1_score(y_val, val_pred, average='weighted')

print("\n[VAL RESULTS]")
print(f"Accuracy     : {val_acc:.4f}")
print(f"F1 (macro)   : {val_f1_macro:.4f}")
print(f"F1 (weighted): {val_f1_weighted:.4f}")


[VAL RESULTS]
Accuracy     : 0.6608
F1 (macro)   : 0.5788
F1 (weighted): 0.6560


In [7]:
# Test prediction
test_pred = mlp.predict(X_test)
test_proba = mlp.predict_proba(X_test)

# Accuracy
test_acc = accuracy_score(y_test, test_pred)

# F1 scores
test_f1_macro = f1_score(y_test, test_pred, average='macro')
test_f1_micro = f1_score(y_test, test_pred, average='micro')
test_f1_weighted = f1_score(y_test, test_pred, average='weighted')

# ROC-AUC (macro)
test_roc_auc = roc_auc_score(y_test, test_proba, multi_class='ovr', average='macro')

# PR-AUC (macro)
pr_aucs = []
for c in range(test_proba.shape[1]):
    y_true_binary = (y_test == c).astype(int)
    prec, rec, _ = precision_recall_curve(y_true_binary, test_proba[:, c])
    pr_aucs.append(auc(rec, prec))
test_pr_auc_macro = np.mean(pr_aucs)

# PRINT
print("\n========== [TEST RESULTS] ==========")
print(f"Accuracy        : {test_acc:.4f}")
print(f"F1 (macro)      : {test_f1_macro:.4f}")
print(f"F1 (micro)      : {test_f1_micro:.4f}")
print(f"F1 (weighted)   : {test_f1_weighted:.4f}")
print(f"ROC-AUC (macro) : {test_roc_auc:.4f}")
print(f"PR-AUC (macro)  : {test_pr_auc_macro:.4f}")

print("\nClassification Report :")
print(classification_report(y_test, test_pred, digits=4))


========== [TEST RESULTS] ==========
Accuracy        : 0.6697
F1 (macro)      : 0.5930
F1 (micro)      : 0.6697
F1 (weighted)   : 0.6656
ROC-AUC (macro) : 0.9779
PR-AUC (macro)  : 0.6192

Classification Report :
              precision    recall  f1-score   support

           0     0.6415    0.5667    0.6018        60
           1     0.6769    0.7333    0.7040        60
           2     0.5294    0.6000    0.5625        60
           3     0.5455    0.4000    0.4615        60
           4     0.7209    0.5167    0.6019        60
           5     0.4118    0.3500    0.3784        60
           6     0.6338    0.7500    0.6870        60
           7     0.6750    0.4500    0.5400        60
           8     0.4096    0.5667    0.4755        60
           9     0.6216    0.3833    0.4742        60
          10     0.6000    0.5000    0.5455        60
          11     0.7073    0.4833    0.5743        60
          12     0.8000    0.6000    0.6857        60
          13     0.2593    0.2